In [1]:
#Libs necessárias
import spacy
import PyPDF2
import nltk
import pandas as pd
from time import time 
from collections import defaultdict
import re
import unidecode
import numpy as np
import os
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import multiprocessing
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors
from calendar import month_name
from collections import Counter
#import nltk
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)


In [2]:
#Support functions

import en_core_web_md
nlp = en_core_web_md.load()
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
stops = list(spacy_stopwords)
months = [m.lower() for m in month_name[1:]]
stopw = stops + months

def sub_str(x): 

    lista=[]

    for x in x.split():

        #x_1   = re.sub("([\d]+) *th",'',x)
        x_2=re.sub(r'[^a-zA-Z0-9]','',x)
        x_new = re.sub("\d+",'',x_2)
        #x_new = re.sub(r'(\d+)\s*(?:hours?\b|h?\b)','',x_)   
        lista+=[x_new]

    alpha_only_string = " ".join(lista)

    return alpha_only_string

#Cleaning functions

def split(x):

    if '\r\n' in x:

        x=x.split('\r\n')
        ''.join(x)
    else:

        x=x.split('\n')

        if '' in x:

            x.remove('')

    return ''.join(x)


#print(" ".join([ent.text for ent in doc if not ent.ent_type_]))
def clean_words(t):
    x = []
    for w in t.split():
        if (len(w) > 2 and w not in stopw):
            x+=[w]
    z = ' '.join(x)
    return z

def clean_ents(t):
    doc = nlp(t)
    return " ".join([ent.text for ent in doc if not ent.ent_type_])

#More cleaning



In [3]:
#Load csv file containing economic news
df = pd.read_csv('Full-Economic-News-DFE-839861.csv',encoding='latin1')
print(df.shape)
df.head()

(8000, 15)


,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,positivity,positivity:confidence,relevance,relevance:confidence,articleid,date,headline,positivity_gold,relevance_gold,text
0,842613455,False,finalized,3,12/5/15 17:48,3.0,0.6400,yes,0.640,wsj_398217788,8/14/91,Yields on CDs Fell in the Latest Week,NaN,NaN,NEW YORK -- Yields on most certificates of dep...
1,842613456,False,finalized,3,12/5/15 16:54,NaN,NaN,no,1.000,wsj_399019502,8/21/07,The Morning Brief: White House Seeks to Limit ...,NaN,NaN,The Wall Street Journal Online</br></br>The Mo...
2,842613457,False,finalized,3,12/5/15 1:59,NaN,NaN,no,1.000,wsj_398284048,11/14/91,Banking Bill Negotiators Set Compromise --- Pl...,NaN,NaN,WASHINGTON -- In an effort to achieve banking ...
3,842613458,False,finalized,3,12/5/15 2:19,NaN,0.0000,no,0.675,wsj_397959018,6/16/86,Manager's Journal: Sniffing Out Drug Abusers I...,NaN,NaN,The statistics on the enormous costs of employ...
4,842613459,False,finalized,3,12/5/15 17:48,3.0,0.3257,yes,0.640,wsj_398838054,10/4/02,Currency Trading: Dollar Remains in Tight Rang...,NaN,NaN,NEW YORK -- Indecision marked the dollar's ton...


In [4]:
#Excluindo categorias com pouca frequência
#1-negative and 9-positive

#list_index1=list(df[df['positivity']==9].index)

#df.drop(df.index[list_index1],inplace=True)

#df=df.reset_index(drop=True)

df['positivity'].value_counts()

3.0    343
7.0    295
4.0    255
6.0    214
5.0    205
8.0     71
2.0     35
9.0      2
Name: positivity, dtype: int64

In [5]:
df.isnull().sum()

_unit_id                    0
_golden                     0
_unit_state                 0
_trusted_judgments          0
_last_judgment_at           0
positivity               6580
positivity:confidence    4225
relevance                   0
relevance:confidence        0
articleid                   0
date                        0
headline                    0
positivity_gold          8000
relevance_gold           8000
text                        0
dtype: int64

In [6]:
#Remove empty columns
df.drop(['positivity:confidence','positivity_gold','positivity_gold','relevance_gold'],axis=1,inplace=True)
df.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,positivity,relevance,relevance:confidence,articleid,date,headline,text
0,842613455,False,finalized,3,12/5/15 17:48,3.0,yes,0.640,wsj_398217788,8/14/91,Yields on CDs Fell in the Latest Week,NEW YORK -- Yields on most certificates of dep...
1,842613456,False,finalized,3,12/5/15 16:54,NaN,no,1.000,wsj_399019502,8/21/07,The Morning Brief: White House Seeks to Limit ...,The Wall Street Journal Online</br></br>The Mo...
2,842613457,False,finalized,3,12/5/15 1:59,NaN,no,1.000,wsj_398284048,11/14/91,Banking Bill Negotiators Set Compromise --- Pl...,WASHINGTON -- In an effort to achieve banking ...
3,842613458,False,finalized,3,12/5/15 2:19,NaN,no,0.675,wsj_397959018,6/16/86,Manager's Journal: Sniffing Out Drug Abusers I...,The statistics on the enormous costs of employ...
4,842613459,False,finalized,3,12/5/15 17:48,3.0,yes,0.640,wsj_398838054,10/4/02,Currency Trading: Dollar Remains in Tight Rang...,NEW YORK -- Indecision marked the dollar's ton...


In [8]:
lista_class = []

for i in df['positivity']:
    if i < 6:
        lista_class+=[0] #negative
    else:
        lista_class+=[1] #positive
df['class'] = pd.DataFrame(lista_class)

In [9]:
df['class'].value_counts()

1    7162
0     838
Name: class, dtype: int64

In [10]:
df=df.dropna()
print(df.shape)
df.isnull().sum()

(1420, 13)


_unit_id                0
_golden                 0
_unit_state             0
_trusted_judgments      0
_last_judgment_at       0
positivity              0
relevance               0
relevance:confidence    0
articleid               0
date                    0
headline                0
text                    0
class                   0
dtype: int64

In [11]:
#Cleaning data and manipulation
d = df.copy()
d['text'][5]

'Stocks declined, as investors weighed slower-than-expected domestic economic growth and continued euro-zone concerns against signs the Federal Reserve may take new steps to bolster the economy.</br></br>The Dow Jones Industrial Average fell 53.59 points, or 0.5%, to 11493.72, its lowest close since Oct. 17.</br></br>Weighing on the downside were Alcoa, which dropped 21 cents, or 2.2%, to $9.26, and Bank of America, which fell 12 cents, or 2.2%, to 5.37. Hewlett-Packard dropped 21 cents, or 0.8%, to 26.65, after the technology company issued a downbeat earnings outlook for the current quarter and the next fiscal year.</br></br>The S&P 500-stock index fell 4.94 points, or 0.4%, to 1188.04.</br></br>The Nasdaq Composite lost 1.86 points, or 0.1%, to 2521.28.'

In [12]:
d['text'] = d['text'].apply(lambda x:x.lower())
d['text'] = d['text'].str.replace('</br>',' ')
d['text'] = d['text'].str.replace('-',' ')

#Remove ponctuation
exclude = set(string.punctuation)
d['text'] = d['text'].apply(lambda texto:''.join(ch for ch in texto if ch not in exclude))
d['text'] = d['text'].apply(lambda x:clean_ents(x))
d['text'] = d['text'].apply(lambda x:sub_str(x))
d['text'] = d['text'].apply(lambda x:clean_words(x))
d = d.reset_index(drop=True)

In [13]:
#Count most frequent words in dataset
Counter(" ".join(d["text"]).split()).most_common(30)

[('said', 1739),
 ('market', 1317),
 ('economy', 1060),
 ('rates', 994),
 ('rate', 966),
 ('prices', 964),
 ('economic', 905),
 ('stock', 880),
 ('interest', 846),
 ('inflation', 741),
 ('growth', 727),
 ('rose', 635),
 ('investors', 616),
 ('index', 616),
 ('average', 607),
 ('stocks', 590),
 ('dollar', 555),
 ('new', 517),
 ('points', 503),
 ('fell', 481),
 ('bond', 473),
 ('sales', 438),
 ('consumer', 437),
 ('higher', 435),
 ('industrial', 433),
 ('markets', 423),
 ('trading', 403),
 ('spending', 403),
 ('report', 397),
 ('dow', 396)]

In [14]:
#Features para criação do modelo preditivo

features = d[['articleid','text']]

labels = d['class']

print('Tamanho da base de features:',features.shape)

print('Tamanho da base de labels:',labels.shape)

Tamanho da base de features: (1420, 2)
Tamanho da base de labels: (1420,)


In [15]:
#Pacotes para Vetorização dos Textos e Machine Learning (varios testes para modelos distintos)

import sklearn

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import cross_val_score

from sklearn import metrics

from sklearn.metrics import classification_report

from sklearn.pipeline import make_pipeline

from sklearn import model_selection

from sklearn.metrics import confusion_matrix

from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier

from sklearn.pipeline import Pipeline

from sklearn.metrics import fbeta_score, make_scorer

from sklearn.metrics import precision_score

from sklearn.metrics import accuracy_score

from sklearn.metrics import recall_score

from sklearn.metrics import f1_score

from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.model_selection import cross_val_score

 

In [16]:
X_train, X_test, y_train, y_test = train_test_split(features,labels, test_size=0.33, random_state=42)

print(X_train.shape)

print(y_train.shape)

(951, 2)
(951,)


In [17]:
#stopwords
stopw += ['said','president','wall','street','standard','poors','ll','ve','vice','dow','jones','nasdaq','th',
          'chairman', 'chief', 'executive','new','york','analysts','federal','investors','economist','government',
          'economists','dollar','point','stock','time','stocks','points','bank','bond','nations','traders','face',
          'times','yields','generally','data','meeting','committee','figures','director','chairman','going','home','copom',
          'session','people','public','according','blue','chip','feds','banks','area','city','says','officials','day','secretary',
          'report','showed','todays','health','middle','class','crude','oil','focus','survey','ex','ante']


In [30]:
#Pipeline com aplicação do Modelo Preditivo - LogReg

pipeline = Pipeline([

    ('tfidf', TfidfVectorizer(stop_words=stopw,strip_accents='unicode',ngram_range=(2,4),

    max_features=300)),

    ('clf',LogisticRegression(penalty='l1', C = 0.72))])

 

#Média e desvio padrão do modelo usando cross-validation

score_nb=cross_val_score(pipeline,X_train['text'],y_train,cv=StratifiedShuffleSplit(n_splits=5, test_size=0.33, random_state=42),scoring='f1_micro')

print(score_nb.mean())

print(score_nb.std())



C:\Users\veren\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.592356687898089
0.01139397695541294


In [31]:
# Fit do pipeline

pipeline.fit(X_train['text'], y_train)

from sklearn.metrics import f1_score

print(f1_score(y_test, pipeline.predict(X_test['text']), average = 'micro'))
#print(accuracy_score(y_test,pipeline.predict(X_test['text'])))

0.5842217484008528


In [32]:
#Checking LR coefs
coefs = pd.DataFrame({"coef":pipeline['clf'].coef_[0],"word":pipeline['tfidf'].get_feature_names()}).sort_values("coef", 
ascending=False)#.query("coef!=0")
coefs

,coef,word
137,1.821391,industrial average rose
208,1.070785,personal income
162,0.922998,labor market
23,0.919383,bull market
132,0.769482,index rose
...,...,...
57,-0.408530,deficit reduction
91,-0.509134,financial crisis
246,-0.652523,record high
129,-1.429050,index fell


In [39]:
#pd.DataFrame({"coef":pipeline['clf'].coef_[0],"word":pipeline['tfidf'].get_feature_names()}).sort_values("coef", 
#ascending=False).query("coef!=0")

In [33]:
#Verificando o vocabulário para classificação

#pipeline['tfidf'].vocabulary_

In [34]:
import collections

#Adicionando palavras que considero relevantes para classificação

vocabulary={'growth slowdown':300,
            'causing significant slowdown':301,
         'remains challenging':302,
         'sharp drop':303,
         'historically low levels':304,
         'supply shock':305,
         'economic recovery delayed':306,
         'remains uncertain': 307,
         'economic contraction':308,
         'uncertainty arises':309,
         'favorably expected signs':310,
         'consistent stabilization':311,
         'gradual recovery':312,
         'favorable global economic activity':313,
         'consistent recovery':314,
         'prospects economic recovery':315,
         'anchored expectations':316,
         'frustration expectations':317,
         'evolved favorably':318,
         'benign evolution':319,
         'gradual normalization':320,
         'levels comfortable':321,
         'conditions favorable':322,
         'easing process continue':323,
         'employment gains':324, 'recovery advances':325, 'favorable evolution':326, 'consumption growth':327,
         'stronger recovery':328,'evolved expected':329,'lower global growth':330,'benign behavior':331, 'comfortable levels':332
    
}

 

d=pipeline['tfidf'].vocabulary_

od = collections.OrderedDict(sorted(d.items(),reverse=False,key=lambda x:x[0]))

n=len(od)

for feature, index in vocabulary.items():

    if feature not in od.values():

        od[feature] = n + index

#Final vocab

final_vocab = {key: v for key,v in zip(od,range(0,len(od)))}

In [35]:
dict_weights = {
            'growth slowdown':-0.5,
            'causing significant slowdown':-0.5,
         'remains challenging':-0.2,
         'sharp drop':-0.7,
         'historically low levels':-1,
         'supply shock':-0.5,
         'economic recovery delayed':-0.5,
         'remains uncertain': -0.5,
         'economic contraction':-1,
         'uncertainty arises':-0.5,
         'favorably expected signs':2,
         'consistent stabilization':1,
         'gradual recovery':1,
         'favorable global economic activity':1,
         'consistent recovery':1,
         'prospects economic recovery':1,
         'anchored expectations':1,
         'frustration expectations':-0.5,
         'evolved favorably':2,
         'benign evolution':2,
         'gradual normalization':1.5,
         'levels comfortable':2,
         'conditions favorable':2,
         'easing process continue':1,'employment gains':2, 'recovery advances':2, 'favorable evolution':1, 'consumption growth':2,
         'stronger recovery':2,'evolved expected':1,'lower global growth':-1,'benign behavior':1, 'comfortable levels':1
    
}
#Append weights to coefs from LR
for x,y in zip(list(dict_weights.keys()),list(dict_weights.values())):
    
    if x in list(coefs['word']):
        coefs.loc[coefs['word'] == x]['coef'].values[0] = y
    
    else:
        coefs = coefs.append({'coef':y,'word':x},ignore_index=True)
coefs

,coef,word
0,1.821391,industrial average rose
1,1.070785,personal income
2,0.922998,labor market
3,0.919383,bull market
4,0.769482,index rose
...,...,...
327,2.000000,stronger recovery
328,1.000000,evolved expected
329,-1.000000,lower global growth
330,1.000000,benign behavior


In [36]:
pipeline_ = Pipeline([

    ('tfidf', TfidfVectorizer(stop_words=stopw,strip_accents='unicode',ngram_range=(2,3),

    vocabulary = final_vocab)),

    ('clf',LogisticRegression(penalty='l1', C = 0.72))])

list_coefs = np.array([coefs['coef'].to_list()])
#Média e desvio padrão do modelo usando cross-validation

score_nb=cross_val_score(pipeline_,X_train['text'],y_train,cv=StratifiedShuffleSplit(n_splits=5, test_size=0.33, random_state=42),scoring='f1_micro')

print(score_nb.mean())

print(score_nb.std())



C:\Users\veren\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.5936305732484076
0.011500299417368015


In [37]:
# Fit do pipeline

pipeline_.fit(X_train['text'], y_train)

from sklearn.metrics import f1_score

print(f1_score(y_test, pipeline_.predict(X_test['text']), average = 'micro'))
print(accuracy_score(y_test,pipeline_.predict(X_test['text'])))

0.5842217484008528
0.5842217484008528


In [38]:
#Setting new coefficients
pipeline_['clf'].coef_ = list_coefs

In [39]:
print(f1_score(y_test, pipeline_.predict(X_test['text']), average = 'micro'))
print(accuracy_score(y_test,pipeline_.predict(X_test['text'])))

0.6012793176972282
0.6012793176972282


In [40]:
#Checking LR coefs
pd.DataFrame({"coef":pipeline_['clf'].coef_[0],"word":pipeline_['tfidf'].get_feature_names()}).sort_values("coef", 
ascending=False).query("coef!=0")

,coef,word
320,2.000000,levels comfortable
324,2.000000,recovery advances
317,2.000000,evolved favorably
318,2.000000,benign evolution
321,2.000000,conditions favorable
...,...,...
303,-1.000000,historically low levels
307,-1.000000,economic contraction
329,-1.000000,lower global growth
298,-1.429050,wholesale prices


In [41]:
#Teste nos textos do BC
d2 = pd.read_excel('cleaned_data.xlsx')
dv = pd.read_excel('base_val.xlsx')
d2 = d2.dropna()
d2.head()

,Unnamed: 0,Name,Text,EconOutlook,Risks_Scenario,MonetaryPol,MonetPolDec
0,0,copom 229 minutes,th update economic outlook global outlook nove...,global outlook novel coronavirus pandemic caus...,encompasses risk factors directions hand econo...,discussed evolution economic activity light av...,lower selic rate percentage judges decision re...
1,1,copom minutes 228,update economic outlook economic activity rele...,economic activity released previous indicate c...,encompasses risk factors directions hand high ...,discussed evolution economic activity light av...,lower selic rate percentage judges decision re...
2,2,copom minutes 230,update economic outlook scenario global outloo...,global outlook covid pandemic causing signific...,inflation encompasses risk factors directions ...,global economy judges previous international c...,lower selic rate percentage judges decision re...
3,3,copom_aug_5th,lowers selic rate lowers selic rate august pub...,following observations provide update global o...,prospectively uncertainty economic growth rema...,believes persevering process reforms necessary...,consequently possible future adjustments curre...
4,4,min2017204th-copom20170117-204th_copom_minutes,thcopommeetinga update economic outlook scenar...,set indicators released recently suggests weak...,risks inflation short run inflation behavior f...,discussed evolution economic activity light re...,reduce selic rate year bias judges convergence...


In [46]:
d2['MonetaryPol'][25]

'global economy available q gdp surprise depth current economic downturn comparable great depression promising signs recovery shown incomplete strong recovery consumption followed services sector pondered main restriction recovery major economies evolution pandemic possibility wave virus major risk discussed evolution domestic economic activity light available information indicators evaluated recent suggest partial recovery economic activity transfer programs allowed relatively strong resumption durable goods consumption investment service sector activities especially directly affected social distancing measures remain depressed looking forward poor predictability associated evolution pandemic necessary decline emergency aid value obtained usual procedure rounding average rus exchange rate observed ended prior nd increase uncertainty speed economic recovery considered unpredictability risks associated evolution pandemic imply domestic characterized gradual recovery economy debated leve

In [47]:
pipeline_.predict(d2['MonetaryPol'])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       1, 0], dtype=int64)